## The Challenge

When you offer funds in multiple currencies or with multiple share classes it can be challenging to ensure that all funds are aligned. This leads to the potential risk that some funds may drift from the core mandate of the original base fund. In addition it leads to wasted time and resources in keeping the funds in-sync with each other.

## The Solution

Rather than maintaining a separate fund for each currency or share class and spending time ensuring that they all reconcile with each other, LUSID allows you to create a single base fund to hold the core transactions that generate the fund's returns.

Your funds offered in additional currencies can hold units in this core fund and any transactions that occur in the base fund will be propogated to your additional currency funds.

Furthermore you can then apply appropriate hedging activity on each additional currency fund without worrying about it affecting the base fund in any way.

In this tutorial you will walk through how to set this up from a completely blank LUSID environment by:

1) Setting up a Scope to Hold your Portfolio

2) Creating your Instrument Universe 

3) Creating your base fund portfolio

4) Setting your Initial Holdings

5) Valuing your base fund

6) Securitising your base fund

7) Creating your foreign currency portfolios

8) Adding units of the base fund to your currency portfolios

9) Adding appropriate hedging transactions for each foreign currency fund

10) Valuing your foreign currency funds

11) Adding additional transactions to the base fund

12) Re-valuing the base fund

13) Re-valuing the foreign currency funds

*Run the cell below to import the libraries and authenticate your LUSID client*

In [1]:
# Import LUSID
import lusid.models as models
import lusid_sample_data as import_data

# Import Libraries
import pprint
from datetime import datetime, timedelta, time
from dateutil.parser import parse
import pytz
import printer as prettyprint
import pandas as pd
import uuid
import math
import json

# Authenticate our user and create our API client
client = import_data.authenticate_secrets()

print ('LUSID Environment Initialised')
print ('API Version: ', client.metadata.get_lusid_versions().build_version)

LUSID Environment Initialised
API Version:  0.5.2862.0


![Initailise LUSID](img/multiplecurrencies-initialise.gif)

## 1) Set up your Scope

You will need to create a name for the scope that you will use for the fund. Read more about scopes in the [LUSID Knowledge Base: Scopes](https://support.lusid.com/what-is-a-scope-in-lusid-and-how-is-it-used).

*Run the cell below to create a name for your scope*

In [2]:
# Give the scope a unique identifier
scope_id = import_data.create_scope_id()
# Give the scope a descriptive name
scope_name = 'UK_High_Growth_Equities_Fund'
# Join the two together to get the full scope name
scope = '{}_{}'.format(scope_name, scope_id)
prettyprint.heading('Scope', scope)

Scope: UK_High_Growth_Equities_Fund_3785-441c-0699-68


![Scope](img/multiplecurrencies-scope.gif)

## 2) Load Instruments

Before you can take on any holdings or make any trades you need to ensure that your instrument universe has been populated. In this case you will import your instrument universe from a CSV file. Read more about instruments in LUSID in the [LUSID Knowledge Base: Instruments](https://support.lusid.com/instruments).

*Run the cell below to import your instrument universe*

In [3]:
# Import your instrument universe from a CSV file
instrument_universe = pd.read_csv('data/multiplecurrencies-instruments.csv')
# Look at the first 10 instruments
instrument_universe.head(n=10)

,instrument_name,client_internal,currency,isin,figi,exchange_code,country_issue,ticker,market_sector,security_type,coupon
0,Amazon_Nasdaq_AMZN,imd_34634534,USD,US0231351067,BBG000BVPXP1,UN,united_states_america,AMZN,equity,common_stock,NaN
1,Apple_Nasdaq_AAPL,imd_35345345,USD,US0378331005,BBG000B9XVV8,UN,united_states_america,AAPL,equity,common_stock,NaN
2,USTreasury_2.00_2021,imd_34535347,USD,US912828U816,BBG00FN3B5K8,BERLIN,united_states_america,T 2 12/31/21,govt,us_government,2.000
3,USTreasury_6.875_2025,imd_34534539,USD,US912810EV62,BBG000DQQNJ8,NEW YORK,united_states_america,T 6.875 08/15/25,govt,us_government,6.875
4,ExpressScripts_NYSE_ESRX,imd_34352311,USD,US30219G1085,BBG000C16621,UN,united_states_america,ESRX,equity,common_stock,NaN
5,TrinityIndustries_NYSE_TRN,imd_34235200,USD,US8965221091,BBG000BVL406,UN,united_states_america,TRN,equity,common_stock,NaN
6,Trex_NYSE_TREX,imd_32423956,USD,US89531P1057,BBG000BTGM43,UN,united_states_america,TREX,equity,common_stock,NaN
7,Cigna_NYSE_CI,imd_32452391,USD,US1255091092,BBG00KXXK940,UN,united_states_america,CI,equity,common_stock,NaN
8,Arcosa_NYSE_ACA,imd_23423409,USD,US0396531008,BBG00JGMWFQ5,UN,united_states_america,ACA,equity,common_stock,NaN


Now that you have the details for your instruments you can go ahead and create an instrument definition for each instrument. These can then be upserted into LUSID. Read about instrument definitions here [LUSID Knowledge Base: What is an Instrument?](https://support.lusid.com/what-is-an-instrument)

As part of this definition you will attach identifiers to your instruments. Read more about identifiers here [LUSID Knowledge Base: Which Instrument Identifier Schemes Should I Use With LUSID?](https://support.lusid.com/which-instrument-identifier-schemes-should-i-use-with-lusid)

You use an upsert method to add instrument definitions to the instrument universe in LUSID. Read more about the behaviour of the upsert method here [LUSID Knowledge Base: Upsert](https://support.lusid.com/upsert-command).

For further usage of the upsert instruments API call refer to the [LUSID API Docs: Upsert Instruments](https://docs.lusid.com/#operation/UpsertInstruments).

*Run the cell below to upsert your instruments into LUSID*

In [4]:
# Initialise a dictionary to hold your instrument definitions
instrument_definitions = {}

# Set the mapping between your identifier columns in the CSV and the available LUSID identifiers
identifier_columns = {
    'isin': 'Isin',
    'figi': 'Figi',
    'ticker': 'Ticker',
    'client_internal': 'ClientInternal'
}

# Iterate over your instrument universe
for index, instrument in instrument_universe.iterrows():

    # Initialise your set of identifiers for this instrument
    identifiers = {}
    
    # Populate your set of identifiers
    for identifier_column, identifier_lusid in identifier_columns.items():
        identifiers[identifier_lusid] = models.InstrumentIdValue(
            value=instrument[identifier_column])
        
    # Create the definition for your instrument
    instrument_definitions[instrument['instrument_name']] = models.InstrumentDefinition(
        name=instrument['instrument_name'],
        identifiers=identifiers
    )

# Call LUSID to upsert your instrument defintions as a batch
response = client.instruments.upsert_instruments(
    requests=instrument_definitions)

# Pretty print the response from LUSID
prettyprint.instrument_response(response, identifier='Figi')

Instrument Successfully Upserted: Amazon_Nasdaq_AMZN
Figi ID: BBG000BVPXP1
LUSID Instrument ID: LUID_5LGL3KBC


Instrument Successfully Upserted: USTreasury_2.00_2021
Figi ID: BBG00FN3B5K8
LUSID Instrument ID: LUID_YWK3GYTI


Instrument Successfully Upserted: Cigna_NYSE_CI
Figi ID: BBG00KXXK940
LUSID Instrument ID: LUID_KI616A6T


Instrument Successfully Upserted: USTreasury_6.875_2025
Figi ID: BBG000DQQNJ8
LUSID Instrument ID: LUID_52T5E6KZ


Instrument Successfully Upserted: Arcosa_NYSE_ACA
Figi ID: BBG00JGMWFQ5
LUSID Instrument ID: LUID_T44OEB6L


Instrument Successfully Upserted: ExpressScripts_NYSE_ESRX
Figi ID: BBG000C16621
LUSID Instrument ID: LUID_K38WBDIV


Instrument Successfully Upserted: TrinityIndustries_NYSE_TRN
Figi ID: BBG000BVL406
LUSID Instrument ID: LUID_XDOYC25R


Instrument Successfully Upserted: Apple_Nasdaq_AAPL
Figi ID: BBG000B9XVV8
LUSID Instrument ID: LUID_N59AHAVB


Instrument Successfully Upserted: Trex_NYSE_TREX
Figi ID: BBG000BTGM43
LUSID Instrument ID: LU

![Instruments](img/multiplecurrencies-instruments.gif)

## 3) Create your Base Fund Portfolio

To create a portfolio you need to give it a name and code.

*Run the cell below to give your base fund portfolio a name and code*

In [5]:
# Give your portfolio a name & code
base_portfolio_name = '{}_base_fund'.format(scope_name)
base_portfolio_code = str(uuid.uuid4())
prettyprint.heading('Portfolio Name', base_portfolio_name)
prettyprint.heading('Portfolio Code', base_portfolio_code)

Portfolio Name: UK_High_Growth_Equities_Fund_base_fund
Portfolio Code: 59a1e19a-21be-4346-be42-cb7303ad6dc0


Now that you have decided on the name and unique code for your portfolio you can create your base fund. Read more about portfolios in the [LUSID Knowledge Base: Portfolios](https://support.lusid.com/portfolios).

For further usage of the create portfolio API call refer to the [LUSID API Docs: Create Portfolio](https://docs.lusid.com/#operation/CreatePortfolio).

Note that when you create the portolio in the cell below you are creating it with a 'created' date of 365 days ago. This number is rather arbitary, in practice it should be the date the portfolio came into existence regardless of the system you first created it in, read more about the importance of the created date on a portfolio in the [LUSID Knowledge Base: Importance of Portfolio Creation Date](https://support.lusid.com/importance-of-portfolio-creation-date).

*Run the cell below to create your portfolio*

In [6]:
# The date your portfolio was first created
portfolio_creation_date = (datetime.now(pytz.UTC) - timedelta(days=365))

# Create the request to add your portfolio to LUSID
transaction_portfolio_request = models.CreateTransactionPortfolioRequest(
    display_name=base_portfolio_name,
    code=base_portfolio_code,
    base_currency='USD',
    description='The portfolio to hold our base fund',
    created=portfolio_creation_date)

# Call LUSID to create your portfolio
response = client.transaction_portfolios.create_portfolio(
    scope=scope,
    create_request=transaction_portfolio_request)

# Pretty print the response from LUSID
prettyprint.portfolio_response(response)

Portfolio Created
Scope: UK_High_Growth_Equities_Fund_3785-441c-0699-68
Code: 59a1e19a-21be-4346-be42-cb7303ad6dc0
Portfolio Effective From: 2018-07-10 13:25:37.980133+00:00
Portfolio Created On: 2019-07-10 13:25:38.789131+00:00



![Base-Fund](img/multiplecurrencies-basefundportfolio.gif)

## 4) Set your Holdings

Now that you have your instrument universe populated and base fund portfolio you can load your initial holdings into your base fund. In this case you will import your holdings from a CSV file.

*Run the cell below to import your take on balances*

In [7]:
#Import your holdings
holdings = pd.read_csv('data/multiplecurrencies-holdings.csv')
holdings.head()

,instrument_name,client_internal,isin,figi,quantity,unit_cost,total_cost,currency
0,Amazon_Nasdaq_AMZN,imd_34634534,US0231351067,BBG000BVPXP1,5000,1550.00,7750000.00,USD
1,Apple_Nasdaq_AAPL,imd_35345345,US0378331005,BBG000B9XVV8,49567,190.00,9417730.00,USD
2,USTreasury_2.00_2021,imd_34535347,US912828U816,BBG00FN3B5K8,121543,99.25,12063142.75,USD
3,USTreasury_6.875_2025,imd_34534539,US912810EV62,BBG000DQQNJ8,98444,140.98,13878635.12,USD


You can add these holdings to LUSID by setting the holdings on your base fund. Read more about how making an adjustment or setting the holdings on a portfolio affects it here [LUSID Knowledge Base: The Effect of Holding Adjustments](https://support.lusid.com/how-do-holding-adjustments-affect-a-portfolio). These holdings will be effective as of 4 days ago. This gives you some room to add recent transactions and conduct valuations.

For further usage of the set holdings API call refer to the [LUSID API Docs: Set Holdings](https://docs.lusid.com/#operation/SetHoldings).

*Run the cell below to set the holdings on your base fund*

In [8]:
# Initialise a list to hold your holding adjustments
holding_adjustments = []

# Set the effective date of these holdings to be 4 days ago
holdings_effective_date = datetime.now(pytz.UTC) - timedelta(days=4)

# Iterate over your holdings
for index, holding in holdings.iterrows():
    
    # Create a holding adjustment for this holding
    holding_adjustments.append(
        models.AdjustHoldingRequest(
            instrument_identifiers={
                    'Instrument/default/Figi': holding['figi']},
                tax_lots=[
                    models.TargetTaxLotRequest(
                        units=holding['quantity'],
                        cost=models.CurrencyAndAmount(
                            amount=holding['total_cost'],
                            currency=holding['currency']),
                        portfolio_cost=holding['total_cost'],
                        price=holding['unit_cost'])
                ]
        )
    )
    
# Call LUSID to set your holdings 
response = client.transaction_portfolios.adjust_holdings(
    scope=scope,
    code=base_portfolio_code,
    effective_at=holdings_effective_date,
    holding_adjustments=holding_adjustments)

# Pretty print the response 
prettyprint.set_holdings_response(response, scope, base_portfolio_code)

Holdings Successfully Set for Portfolio
Scope: UK_High_Growth_Equities_Fund_3785-441c-0699-68
Code: 59a1e19a-21be-4346-be42-cb7303ad6dc0
Holdings Effective From: 2019-07-06 13:25:43.598538+00:00
Holdings Created On: 2019-07-10 13:25:43.878965+00:00



![Base-Fund-Hodlings](img/multiplecurrencies-basefundportfolioholdings.gif)

## 5) Value your Base Fund Portfolio

### a) Upsert Market Quotes

With holdings in your base fund you can now attempt to produce a value for the fund. To value a portfolio in LUSID you need to upsert market data quotes against the underlying holdings or specify an analytics library to use. Read more about aggregating and valuing portfolios in the [LUSID Knowledge Base: Aggregations and Valuations](https://support.lusid.com/what-is-a-valuation). 

In this case you will upsert market data quotes to the quote store to be used in an aggregation request. You will import these quotes from a CSV file.

*Run the cell below to import the market data prices*

In [9]:
# Import market data for the day that the holdings of the fund are effective from
prices = pd.read_csv('data/multiplecurrencies-prices.csv')
prices.head()

,price,type,currency,instrument_name,instrument_figi,instrument_internal
0,1622.65,close,USD,Amazon_Nasdaq_AMZN,BBG000BVPXP1,imd_34634534
1,170.80,close,USD,Apple_Nasdaq_AAPL,BBG000B9XVV8,imd_34535347
2,98.00,close,USD,USTreasury_2.00_2021,BBG00FN3B5K8,imd_34535347
3,142.36,close,USD,USTreasury_6.875_2025,BBG000DQQNJ8,imd_34534539


Now that you have imported the market data you can add it to the quote store in LUSID. Read more about what a quote is in the [LUSID Knowledge Base: What is a Quote?](https://support.lusid.com/what-is-a-quote).

For further usage of the Upsert Quotes API call refer to the [LUSID API Docs: Upsert Quotes](https://docs.lusid.com/#operation/UpsertQuotes).

*Run the cell below to upsert the market data quotes into LUSID*

In [10]:
# Initialise an empty instrument quotes list to hold the quotes
instrument_quotes = []

# Iterate over the quotes
for index, quote in prices.iterrows():
    
    # Add the quote to the list of upsert quote requests
    instrument_quotes.append(
        models.UpsertQuoteRequest(
            quote_id=models.QuoteId(
                provider='DataScope',
                price_source='',
                instrument_id=quote['instrument_figi'],
                instrument_id_type='Figi',
                quote_type='Price',
                price_side='Mid'),
            metric_value=models.MetricValue(
                value=quote['price'],
                unit=quote['currency']),
            effective_at=holdings_effective_date,
            lineage='InternalSystem'
            )
    )

# Upsert the quotes into LUSID
response = client.quotes.upsert_quotes(
    scope=scope,
    quotes=instrument_quotes)

# Pretty print the response
prettyprint.upsert_quotes_response(response)

Quotes Successfully Upserted At: 2019-07-10 13:25:48.369202+00:00


![Analytic-Stores](img/multiplecurrencies-quotestore.gif)

### b) Value using Quotes via Aggregation

Now that the quotes have been added to the quote store you can value your base fund portfolio. You do this in LUSID via an aggregation. The logic for a valuation via aggregation is controled by a LUSID recipe. Read more about recipes in the [LUSID Knowledge Base: What is a Recipe and How Are They Used?](https://support.lusid.com/what-is-a-recipe-and-how-are-they-used).

For further usage of the Get Aggregation by Portfolio API call refer to the [LUSID API Docs: Get Aggregation by Portfolio](https://docs.lusid.com/#operation/GetAggregationByPortfolio).

*Run the cell below to aggregate and value the base fund*

In [11]:
# Specify the inline recipe used to perform the aggregation
inline_recipe = models.ConfigurationRecipe(
    code='quotes_recipe',
    market=models.MarketContext(
        market_rules=[
            models.MarketDataKeyRule(
               key='Equity.Figi.*',
               supplier='DataScope',
               data_scope=scope,
               quote_type='Price',
               price_side='Mid'),
           models.MarketDataKeyRule(
               key='Equity.LusidInstrumentId.*',
               supplier='DataScope',
               data_scope=scope,
               quote_type='Price',
               price_side='Mid'),
            models.MarketDataKeyRule(
               key='Fx.CurrencyPair.*',
               supplier='DataScope',
               data_scope=scope,
               quote_type='Rate',
               price_side='Mid')
        ],
        suppliers=models.MarketContextSuppliers(
            commodity='DataScope',
            credit='DataScope',
            equity='DataScope',
            fx='DataScope',
            rates='DataScope'),
        options=models.MarketOptions(
            default_supplier='DataScope',
            default_instrument_code_type='Figi',
            default_scope=scope)
    )
)

# Create the aggregation request
aggregation_request = models.AggregationRequest(
    inline_recipe=inline_recipe,
    effective_at=holdings_effective_date,
    metrics=[
        models.AggregateSpec(key='Holding/default/SubHoldingKey',
        op='Value'),
        models.AggregateSpec(key='Instrument/default/Name',
        op='Value'),
        models.AggregateSpec(key='Holding/default/Units',
        op='Sum'),
        models.AggregateSpec(key='Holding/default/Cost',
        op='Sum'),
        models.AggregateSpec(key='Holding/default/PV',
        op='Sum'),
    ],
    group_by=[
        'Holding/default/SubHoldingKey'
    ])

# Call LUSID to perform the aggregation
response = client.aggregation.get_aggregation_by_portfolio(
    scope=scope,
    code=base_portfolio_code,
    request=aggregation_request)

# Pretty print the response
prettyprint.aggregation_response_generic(response)

Aggregation Results: 
Holding/default/SubHoldingKey: LusidInstrumentId=LUID_5LGL3KBC/USD
Instrument/default/Name: Amazon_Nasdaq_AMZN
Sum(Holding/default/Units): 5000.0
Sum(Holding/default/Cost): 7750000.0
Sum(Holding/default/PV): 8113250.0


Holding/default/SubHoldingKey: LusidInstrumentId=LUID_N59AHAVB/USD
Instrument/default/Name: Apple_Nasdaq_AAPL
Sum(Holding/default/Units): 49567.0
Sum(Holding/default/Cost): 9417730.0
Sum(Holding/default/PV): 8466043.6


Holding/default/SubHoldingKey: LusidInstrumentId=LUID_YWK3GYTI/USD
Instrument/default/Name: USTreasury_2.00_2021
Sum(Holding/default/Units): 121543.0
Sum(Holding/default/Cost): 12063142.75
Sum(Holding/default/PV): 11911214.0


Holding/default/SubHoldingKey: LusidInstrumentId=LUID_52T5E6KZ/USD
Instrument/default/Name: USTreasury_6.875_2025
Sum(Holding/default/Units): 98444.0
Sum(Holding/default/Cost): 13878635.12
Sum(Holding/default/PV): 14014487.84




![Analytic-Stores](img/multiplecurrencies-aggregatedbase.gif)

## 6) Securitise your Base Fund Portfolio

### a) Securitise with x units

Now that you have created and valued your base fund, you need to securitise it so that it can be held by other portfolios. Before you create the instrument that allows other portfolios to hold the base fund you need a way to hold the total number of units of the fund available to be held.

To do this you can make use of a property. Read more about properties in the [LUSID Knowledge Base: What is a Property?](https://support.lusid.com/what-is-a-property).

For further usage of the Create Property Definition API call refer to the [LUSID API Docs: Create Property Definition](https://docs.lusid.com/#operation/CreatePropertyDefinition).

*Run the cell below to create a property to hold the total circulation of your base fund*

In [12]:
# Create your request to define a new property
property_request = models.CreatePropertyDefinitionRequest(
    domain='Instrument',
    scope=scope,
    code='total_circulation',
    value_required=False,
    display_name='total_circulation',
    data_type_id=models.ResourceId(scope='default', code='number'))

# Call LUSID to create uour new property
response = client.property_definitions.create_property_definition(
    definition=property_request)

# Grab the key off the response to use when referencing this property in other LUSID calls
circulation_property_key = response.key

# Pretty print your key
prettyprint.heading('Circulation Property Key', circulation_property_key)

Circulation Property Key: Instrument/UK_High_Growth_Equities_Fund_3785-441c-0699-68/total_circulation


![Analytic-Stores](img/multiplecurrencies-circulationproperty.gif)

You can now create an securitise the base fund portfolio as an instrument, allowing it to be held by other portfolios. Here you are giving it 40,000 units.

*Run the cell below to securitise the base fund*

In [13]:
# Create the instrument definition for the base fund
instrument_definition = models.InstrumentDefinition(
    name=base_portfolio_name,
    identifiers={'ClientInternal': models.InstrumentIdValue(
        value=base_portfolio_code)},
    properties=[
        models.InstrumentProperty(
            key=circulation_property_key,
            value=models.PropertyValue(
                metric_value=models.MetricValue(
                    value=40000)
            )
        )
    ],
    look_through_portfolio_id=models.ResourceId(
        scope=scope,
        code=base_portfolio_code)
)

# Call LUSID to upser the instrument
response = client.instruments.upsert_instruments(
    requests={
        'securitise-base-fund':instrument_definition
    }
)

# Pretty print the response
prettyprint.instrument_response(response, identifier='ClientInternal')

Instrument Successfully Upserted: securitise-base-fund
ClientInternal ID: 59a1e19a-21be-4346-be42-cb7303ad6dc0
LUSID Instrument ID: LUID_BDD47Y21


1  instruments upserted successfully
0  instrument upsert failures


![Analytic-Stores](img/multiplecurrencies-securitisebase.gif)

### b) Calculalate price from aggregation and store in the quote store

So that you can accurately value any portfolios that hold the newly created base fund instrument, you need to store the value of a unit of the base fund in the quotes store. To do this you can re-value the base fund portfolio as you did earlier. The only difference is that in this case you are grouping all the holdings together to get the total value of the fund.

*Run the cell below to get the total value of the fund*

In [20]:
# Create the aggregation request, using the recipe defined earlier
aggregation_request = models.AggregationRequest(
    inline_recipe=inline_recipe,
    effective_at=holdings_effective_date,
    metrics=[
        models.AggregateSpec(key='Holding/default/Cost',
        op='Sum'),
        models.AggregateSpec(key='Holding/default/PV',
        op='Sum'),
        models.AggregateSpec(key='Portfolio/default/Name',
        op='Value'),
    ],
    group_by=[
        'Portfolio/default/Name'
    ])

# Call LUSID to aggregate and value the base fund
response = client.aggregation.get_aggregation_by_portfolio(
    scope=scope,
    code=base_portfolio_code,
    request=aggregation_request)

# Set the total value and total cost of the base fund when it was first created
base_fund_total_value = response.data[0]['Sum(Holding/default/PV)']
base_fund_total_cost = response.data[0]['Sum(Holding/default/Cost)']

prettyprint.heading("Base Fund Total Value", str(base_fund_total_value))
prettyprint.heading("Base Fund Total Cost", str(base_fund_total_cost))

Base Fund Total Value: 42504995.44
Base Fund Total Cost: 43109507.87


Now that you have the total value of the fund you need to divide this by its total circulation to get the unit price of the base fund. 

*Run the cell below to calculate the unit price of the base fund and upsert this quote into the quotes store*

In [15]:
# Get the base fund instrument
base_fund_instrument = client.instruments.get_instrument(
    identifier_type='ClientInternal',
    identifier=base_portfolio_code,
    instrument_property_keys=[circulation_property_key])

# Get the unique Lusid Instrument Id from the base fund instrument
luid = base_fund_instrument.lusid_instrument_id
# Get the total circulation from the base fund instrument
base_fund_total_circulation = round(base_fund_instrument.properties[0].value, 0)
# Calculate the unit price of the base fund from the total value and the total circulation
base_fund_unit_price = round(base_fund_total_value / base_fund_total_circulation, 2)

# Create a quote for the unit price of the base fund
instrument_quote = models.UpsertQuoteRequest(
    quote_id=models.QuoteId(
        provider='DataScope',
        price_source='',
        instrument_id=luid,
        instrument_id_type='LusidInstrumentId',
        quote_type='Price',
        price_side='Mid'),
    metric_value=models.MetricValue(
        value=base_fund_unit_price,
        unit='USD'),
    effective_at=holdings_effective_date,
    lineage='InternalSystem'
)

# Call LUSID to upsert the quote
response = client.quotes.upsert_quotes(
    scope=scope,
    quotes=[instrument_quote])

# Pretty print the total value, circulation and unit price of the fund
prettyprint.heading('Base Fund Unit Price', str(base_fund_unit_price))
# Pretty print the response from LUSID

prettyprint.upsert_quotes_response(response)

Base Fund Unit Price: 1062.62
Quotes Successfully Upserted At: 2019-07-10 13:25:59.113020+00:00


![Base-Fund-Price](img/multiplecurrencies-basefundprice.gif)

## 7) Create Foreign Currency Portfolios

Now that you've succesfully securitised the base fund and associated it with a unit price, you can create additional portfolios to hold the fund. In this case you want to offer the fund in five different currencies.

Note that each portfolio is created with the appropriate base currency.

*Run the cell below to create a portfolio for each currency you'd like to offer the fund in*

In [16]:
# Define the 5 fund currencies to offer the fund in
fund_currencies = {
    'AUD': None,
    'GBP': None,
    'USD': None,
    'EUR': None,
    'JPY': None
}

# Iterate over each currency
for currency in fund_currencies.keys():
    
    # Create the portfolio name & code for this currency
    portfolio_name = '{}_{}'.format(base_portfolio_name, currency)
    portfolio_code = str(uuid.uuid4())
    fund_currencies[currency] = portfolio_code
    
    # Build the request to create your portfolio in this currency
    transaction_portfolio_request = models.CreateTransactionPortfolioRequest(
        display_name=portfolio_name,
        code=portfolio_code,
        base_currency=currency,
        description='The portfolio to hold our base fund in currency {}'.format(currency),
        created=portfolio_creation_date)

    # Call LUSID to create your portfolio
    portfolio_response = client.transaction_portfolios.create_portfolio(
        scope=scope,
        create_request=transaction_portfolio_request)

    # Pretty print the response from LUSID
    prettyprint.portfolio_response(portfolio_response)

Portfolio Created
Scope: UK_High_Growth_Equities_Fund_3785-441c-0699-68
Code: 95fae232-283d-4d09-8f01-6d3e3142b5bf
Portfolio Effective From: 2018-07-10 13:25:37.980133+00:00
Portfolio Created On: 2019-07-10 13:26:01.385172+00:00

Portfolio Created
Scope: UK_High_Growth_Equities_Fund_3785-441c-0699-68
Code: 8065529b-7d65-422b-b000-263d45c27a89
Portfolio Effective From: 2018-07-10 13:25:37.980133+00:00
Portfolio Created On: 2019-07-10 13:26:01.641127+00:00

Portfolio Created
Scope: UK_High_Growth_Equities_Fund_3785-441c-0699-68
Code: 359e16b8-2b8f-4ec1-86d5-1ee7df57e5e6
Portfolio Effective From: 2018-07-10 13:25:37.980133+00:00
Portfolio Created On: 2019-07-10 13:26:01.844674+00:00

Portfolio Created
Scope: UK_High_Growth_Equities_Fund_3785-441c-0699-68
Code: bd1e6fc5-8e88-496a-9ebd-bed725be0cc1
Portfolio Effective From: 2018-07-10 13:25:37.980133+00:00
Portfolio Created On: 2019-07-10 13:26:02.126768+00:00

Portfolio Created
Scope: UK_High_Growth_Equities_Fund_3785-441c-0699-68
Code: 77

![Foreign-Currency-Portfolios](img/multiplecurrencies-foreigncurrencyportfolios.gif)

## 8) Add Base Fund Units

Now that the portfolios have been created, you can add units of the base fund to them.

To do this you need to know the exchange rate at the time the currency portfolios take on the base fund units. This allows you to calculate the portfolio cost of the holdings.

In this case you will import the foreign exchange rates from a CSV file.

*Run the cell below to import the foreign exchange rates*

In [17]:
# Import the FX rates
fx_rates = pd.read_csv('data/multiplecurrencies-fxrates.csv')
fx_rates.head()

,pair,rate
0,GBP/USD,1.310
1,AUD/USD,0.720
2,EUR/USD,1.140
3,JPY/USD,0.009


Now that you have imported the FX rates you can upsert them into the quote store.

*Run the cell below to upsert the FX rates into the quote store*

In [18]:
# Prepare an empty list to hold the quote requests
quote_requests = []

# Iterate over each FX rate
for index, fx_rate in fx_rates.iterrows():
    
    # Add a quote for the FX rate
    quote_requests.append(
        models.UpsertQuoteRequest(
            quote_id=models.QuoteId(
                provider='DataScope',
                price_source='',
                instrument_id=fx_rate['pair'],
                instrument_id_type='CurrencyPair',
                quote_type='Rate',
                price_side='Mid'),
            metric_value=models.MetricValue(
                value=fx_rate['rate'],
                unit='rate'),
            effective_at=holdings_effective_date,
            lineage='InternalSystem'))

    # Create the reverse pair name 
    reverse_pair = '/'.join(fx_rate['pair'].split('/')[::-1])

    # Add a quote for the reverse FX rate
    quote_requests.append(
        models.UpsertQuoteRequest(
            quote_id=models.QuoteId(
                provider='DataScope',
                price_source='',
                instrument_id=reverse_pair,
                instrument_id_type='CurrencyPair',
                quote_type='Rate',
                price_side='Mid'),
            metric_value=models.MetricValue(
                value=1/fx_rate['rate'],
                unit='rate'),
            effective_at=holdings_effective_date,
            lineage='InternalSystem'))

# Upsert the quotes into LUSID
response = client.quotes.upsert_quotes(
    scope=scope,
    quotes=quote_requests)

# Pretty print the response
prettyprint.upsert_quotes_response(response)

Quotes Successfully Upserted At: 2019-07-10 13:26:06.440705+00:00


Now that you have loaded the FX rates into the quote store you can add units of the securitised base fund to each of the currency portfolios. In this case you will give each currency portfolio the same amount of units.

Note that here you are pulling the FX rates back out of the quote store using the Get Quotes API call. For further usage of the Get Quotes API call refer to the [LUSID API Docs: Get Quotes](https://docs.lusid.com/#operation/GetQuotes).

For further usage of the Adjust Holdings API call refer to the [LUSID API Docs: Adjust Holdings](https://docs.lusid.com/#operation/AdjustHoldings).

*Run the cell below to adjust the holdings of the currency portfolios to hold the base fund*

In [21]:
# Work out the base fund units to give to each portfolio, dividing equally amongst them
base_fund_units_per_portfolio = base_fund_total_circulation/len(fund_currencies)

# Work out the total cost for each portfolio in the currency of the base fund (USD)
total_cost_per_portfolio = base_fund_total_cost/len(fund_currencies)

# Iteratve over each currency
for currency, currency_portfolio_code in fund_currencies.items():
    
    # If the currency is the same as the base fund, skip getting an FX rate
    if currency!='USD':
        # Get the appropriate FX rate from the quotes store
        fx_rate = client.quotes.get_quotes(
            scope=scope,
            quote_ids=[
                models.QuoteId(
                    provider='DataScope',
                    price_source='',
                    instrument_id='{}/USD'.format(currency),
                    instrument_id_type='CurrencyPair',
                    quote_type='Rate',
                    price_side='Mid')
            ],
            effective_at=holdings_effective_date).found[0].metric_value.value
    else:
        # Use an FX rate of 1 if the currency is the same as the base fund
        fx_rate = 1
    
    # Initialise an empty list for the holding adjustments
    holding_adjustments = []
    
    # Add a holding adjustment to the list for this currency portfolio
    holding_adjustments.append(
        models.AdjustHoldingRequest(
            instrument_identifiers={
                    'Instrument/default/ClientInternal': base_portfolio_code},
                tax_lots=[
                    models.TargetTaxLotRequest(
                        units=base_fund_units_per_portfolio,
                        cost=models.CurrencyAndAmount(
                            amount=total_cost_per_portfolio,
                            currency='USD'),
                        portfolio_cost=total_cost_per_portfolio/fx_rate, 
                        price=base_fund_total_cost/base_fund_total_circulation)
                ]
        )
    )

    # Call LUSID to adjust your holdings
    response = client.transaction_portfolios.adjust_holdings(
        scope=scope,
        code=currency_portfolio_code,
        effective_at=holdings_effective_date,
        holding_adjustments=holding_adjustments)
    
    # Pretty print the response from LUSID
    prettyprint.set_holdings_response(response, scope, currency_portfolio_code)

Holdings Successfully Set for Portfolio
Scope: UK_High_Growth_Equities_Fund_3785-441c-0699-68
Code: 95fae232-283d-4d09-8f01-6d3e3142b5bf
Holdings Effective From: 2019-07-06 13:25:43.598538+00:00
Holdings Created On: 2019-07-10 13:26:44.940122+00:00

Holdings Successfully Set for Portfolio
Scope: UK_High_Growth_Equities_Fund_3785-441c-0699-68
Code: 8065529b-7d65-422b-b000-263d45c27a89
Holdings Effective From: 2019-07-06 13:25:43.598538+00:00
Holdings Created On: 2019-07-10 13:26:45.271339+00:00

Holdings Successfully Set for Portfolio
Scope: UK_High_Growth_Equities_Fund_3785-441c-0699-68
Code: 359e16b8-2b8f-4ec1-86d5-1ee7df57e5e6
Holdings Effective From: 2019-07-06 13:25:43.598538+00:00
Holdings Created On: 2019-07-10 13:26:45.477691+00:00

Holdings Successfully Set for Portfolio
Scope: UK_High_Growth_Equities_Fund_3785-441c-0699-68
Code: bd1e6fc5-8e88-496a-9ebd-bed725be0cc1
Holdings Effective From: 2019-07-06 13:25:43.598538+00:00
Holdings Created On: 2019-07-10 13:26:45.748778+00:00



![Foreign-Currency-Portfolios](img/multiplecurrencies-foreigncurrencyportfoliosholdbase.gif)

## 9) Add Hedging Transactions

Even though you are offering your base fund in different currencies, you don't want fluctuations in the exchange rate to affect the performance of each currency portfolio. You therefore plan to hedge against the foreign exchange rate risk. Each hedge comes in the form of a forward fx sell contract which you plan to roll over weekly. 

Read more about creating bespoke instruments in the [LUSID Knowledge Base: Creating Bespoke Instruments](https://support.lusid.com/creating-bespoke-instruments). 

*Run the cell below to create your first hedging contracts*

In [22]:
# Initialise a dictionary to hold the hedging contracts
hedge_contracts = {}

# Iterate over each currency
for currency in fund_currencies.keys():
    
    # If the currency matches the base fund there is no need for a hedge
    if currency == 'USD':
        continue
        
    # Get the current foreign exchange rate for the first hedge contract
    fx_rate = client.quotes.get_quotes(
        scope=scope,
        quote_ids=[
            models.QuoteId(
                provider='DataScope',
                price_source='',
                instrument_id='{}/USD'.format(currency),
                instrument_id_type='CurrencyPair',
                quote_type='Rate',
                price_side='Mid')
        ],
        effective_at=holdings_effective_date).found[0].metric_value.value

    # Create the bespoke hedge contract definition
    hedge_contract = {
        "contract_id": "tid_{}".format(uuid.uuid4()),
        "type": "FwdFxSell",
        "legs": 2,
        "transaction_date": holdings_effective_date.isoformat(),
        "settlement_date": (holdings_effective_date+timedelta(days=7)).isoformat(),
        "strike_price": 1/fx_rate,
        "leg1": {
            "type": "sell",
            "currency": "USD",
            "amount": round(total_cost_per_portfolio, 0)
        },
        "leg2": {
            "type": "buy",
            "currency": currency,
            "amount": round(total_cost_per_portfolio, 0) / fx_rate
        }
    }

     # Create the definition for your instrument, attaching the bespoke contract
    hedge_contracts['{}_hedge'.format(currency)] = models.InstrumentDefinition(
        name='hedge_{}'.format(currency),
        identifiers={
            'ClientInternal': models.InstrumentIdValue(
                value=hedge_contract["contract_id"])},
        definition=models.InstrumentEconomicDefinition(
            instrument_format='JSON',
            content=json.dumps(hedge_contract)
        )
    )
        
# Call LUSID to upsert your instrument defintions
response = client.instruments.upsert_instruments(
    requests=hedge_contracts)

# Pretty print the response from LUSID
prettyprint.instrument_response(response)

Instrument Successfully Upserted: JPY_hedge
ClientInternal ID: tid_f79d13b8-c2de-4212-a115-ab6287d93947
LUSID Instrument ID: LUID_8R4N8DNZ


Instrument Successfully Upserted: GBP_hedge
ClientInternal ID: tid_e42b20e2-ebd1-44af-ac85-90e35bfc9388
LUSID Instrument ID: LUID_VA4MNK5S


Instrument Successfully Upserted: AUD_hedge
ClientInternal ID: tid_a99a1ee6-6143-46f9-937d-3f2d0b344afc
LUSID Instrument ID: LUID_R5VYBQP8


Instrument Successfully Upserted: EUR_hedge
ClientInternal ID: tid_c076dc31-e9bb-43ec-b77e-3e3b1c725f5e
LUSID Instrument ID: LUID_65TL3NNC


4  instruments upserted successfully
0  instrument upsert failures


Now that you've added your hedge contracts to your instrument master inside LUSID you can log transactions against them. Read more about transactions in the [LUSID Knowledge Base: What is a Transaction?](https://support.lusid.com/what-is-a-transaction). 

For further usage of the Upsert Transactions API call refer to the [LUSID API Docs: Upsert Transactions](https://fbn-prd.lusid.com/docs/api#operation/UpsertTransactions).

*Run the cell below to upsert a hedging transaction for each currency portfolio*

In [23]:
# Iterate over the currencies
for currency, currency_portfolio_code in fund_currencies.items():
    
    if currency == 'USD':
        continue
        
    # Initialse a list to hold the hedging transaction
    hedge_transactions = []

    # Retrieve the bespoke hedge contract from the hedging contract definition
    contract = json.loads(client.instruments.get_instrument(
        identifier_type='ClientInternal',
        identifier=hedge_contracts['{}_hedge'.format(currency)].identifiers['ClientInternal'].value
    ).instrument_definition.content)

    # Build and add the hedge transaction for this currency to the list of transactions
    hedge_transactions.append(
        models.TransactionRequest(
            transaction_id=contract['contract_id'],
            type='Buy',
            instrument_identifiers={
                'Instrument/default/ClientInternal': contract['contract_id']},
            transaction_date='T'.join(contract['transaction_date'].split(' ')),
            settlement_date='T'.join(contract['settlement_date'].split(' ')),
            units=1,
            transaction_price=models.TransactionPrice(
                price=0,
                type='Price'),
            total_consideration=models.CurrencyAndAmount(
                amount=0,
                currency=currency),
            transaction_currency='USD',
            source='Client',
        )
    )
    
    # Upsert the transaction into LUSID
    response = client.transaction_portfolios.upsert_transactions(
        scope=scope,
        code=currency_portfolio_code,
        transactions=hedge_transactions
        )
  
    # Pretty print the response from LUSID
    prettyprint.transactions_response(
        response, 
        scope, 
        currency_portfolio_code)

Transactions Successfully Upserted into Portfolio
Scope: UK_High_Growth_Equities_Fund_3785-441c-0699-68
Code: 95fae232-283d-4d09-8f01-6d3e3142b5bf
Transactions Effective From: 2019-07-06 13:25:43.598538+00:00
Transactions Created On: 2019-07-10 13:26:53.559121+00:00

Transactions Successfully Upserted into Portfolio
Scope: UK_High_Growth_Equities_Fund_3785-441c-0699-68
Code: 8065529b-7d65-422b-b000-263d45c27a89
Transactions Effective From: 2019-07-06 13:25:43.598538+00:00
Transactions Created On: 2019-07-10 13:26:53.906386+00:00

Transactions Successfully Upserted into Portfolio
Scope: UK_High_Growth_Equities_Fund_3785-441c-0699-68
Code: bd1e6fc5-8e88-496a-9ebd-bed725be0cc1
Transactions Effective From: 2019-07-06 13:25:43.598538+00:00
Transactions Created On: 2019-07-10 13:26:54.293302+00:00

Transactions Successfully Upserted into Portfolio
Scope: UK_High_Growth_Equities_Fund_3785-441c-0699-68
Code: 77c64321-8e0b-4385-abac-0bd531fe60cb
Transactions Effective From: 2019-07-06 13:25:43.

## 10) Value Currency Portfolios

With the hedging transactions added to each foreign currency portfolio you now want to value them. To do this you first need to value the currency forward contracts.

### a) Value the Currency Forward Contracts

In this case

*Run the cell below to load interest rates from a CSV file*

In [24]:
interest_rates = pd.read_csv('data/multiplecurrencies-interestrates.csv')
interest_rates.head()

,currency,interest_rate
0,USD,0.025500
1,AUD,0.017100
2,EUR,-0.001611
3,JPY,0.001000
4,GBP,0.007719


In [42]:
quote_requests = []

# Iterate over each currency
for currency, currency_portfolio_code in fund_currencies.items():
    
    # Ensure that the currency is not the same as the base fund currency
    if currency!='USD':
        # Get the current FX rate for buying USD from the portfolio currency
        fx_rate = client.quotes.get_quotes(
            scope=scope,
            quote_ids=[
                models.QuoteId(
                    provider='DataScope',
                    price_source='',
                    instrument_id='{}/USD'.format(currency),
                    instrument_id_type='CurrencyPair',
                    quote_type='Rate',
                    price_side='Mid')
            ],
            effective_at=holdings_effective_date).found[0].metric_value.value
    else:
        # If the currency is the same as the base fund use an fx_rate of 1
        fx_rate = 1
    
    # Get all transactions for the currency portfolio
    response = client.transaction_portfolios.get_transactions(
        scope=scope,
        code=currency_portfolio_code,
        from_transaction_date=holdings_effective_date,
        to_transaction_date=holdings_effective_date,
        filter="type eq 'Buy'")
    
    # Ensure that the at least one transaction exists
    if len(response.values) == 0:
        continue
        
    # Get the hedging transaction which should be the only transaction
    hedge_transaction = response.values[0]
    
    # Get the hedge contract using the instrument id from the transaction
    hedge_contract = json.loads(client.instruments.get_instrument(
        identifier_type='LusidInstrumentId',
        identifier=hedge_transaction.instrument_uid
    ).instrument_definition.content)
    
    # Get the spot rate to buy the portfolio currency
    spot_rate = 1 / fx_rate
    
    # Get the interest rate for USD
    interest_rate_1 = interest_rates.loc[
        interest_rates['currency'] == 'USD', 'interest_rate'].values[0]
    
    # Get the interest rate for the portfolio currency
    interest_rate_2 = interest_rates.loc[
        interest_rates['currency'] == currency, 'interest_rate'].values[0]
    
    # Get the time in days left on the hedge contract
    time = ((parse(timestr=hedge_contract['settlement_date']) \
            - holdings_effective_date).days) / 365
    
    # Get the strike price of the hedge contract
    strike_price = hedge_contract['strike_price']

    # Work out the forward FX price
    forward_price = spot_rate * math.exp((interest_rate_2 - interest_rate_1) * time)
    
    # Work out the value of the hedge per unit of currency
    value = (forward_price - strike_price) * math.exp(-interest_rate_2 * time)
    
    # Work out the total value of the hedge
    total_value = value * hedge_contract['leg1']['amount']
    
    # Print the value
    prettyprint.heading(
        'USD/' + currency+' swap value', '{} {}'.format(
            round(total_value,2), currency))
    
    print (hedge_transaction.instrument_uid)
    
    # Upsert the quote against the hedging transaction
    quote_requests.append(
        models.UpsertQuoteRequest(
            quote_id=models.QuoteId(
                provider='DataScope',
                price_source='',
                instrument_id=hedge_transaction.instrument_uid,
                instrument_id_type='LusidInstrumentId',
                quote_type='Price',
                price_side='Mid'),
            metric_value=models.MetricValue(
                value=total_value,
                unit=currency),
            effective_at=holdings_effective_date,
            lineage='InternalSystem'))

response = client.quotes.upsert_quotes(
    scope=scope,
    quotes=quote_requests)

prettyprint.upsert_quotes_response(response)

USD/AUD swap value: -1928.31 AUD
LUID_R5VYBQP8
USD/GBP swap value: -2243.65 GBP
LUID_VA4MNK5S
USD/EUR swap value: -3931.48 EUR
LUID_65TL3NNC
USD/JPY swap value: -450009.29 JPY
LUID_8R4N8DNZ
Quotes Successfully Upserted At: 2019-07-10 13:36:33.969791+00:00


## Value the Currency Portfolios

In [43]:
currency_portfolio_values = {}

inline_recipe_complex = inline_recipe
inline_recipe_complex.pricing = models.PricingContext(
    options=models.PricingOptions(
        allow_any_instruments_with_sec_uid_to_price_off_lookup=True))

for currency, currency_portfolio_code in fund_currencies.items():
    
    aggregation_request = models.AggregationRequest(
        inline_recipe=inline_recipe_complex,
        effective_at=holdings_effective_date,
        metrics=[
            models.AggregateSpec(key='Holding/default/PV',
            op='Sum'),
            models.AggregateSpec(key='Portfolio/default/Name',
            op='Value')
        ],
        group_by=[
            'Portfolio/default/Name'
        ])

    response = client.aggregation.get_aggregation_by_portfolio(
        scope=scope,
        code=currency_portfolio_code,
        request=aggregation_request)

    currency_portfolio_values[currency] = response.data[0]['Sum(Holding/default/PV)']

    prettyprint.aggregation_response_generic(response)

Aggregation Results: 
Sum(Holding/default/PV): 11804960.575389836
Portfolio/default/Name: UK_High_Growth_Equities_Fund_base_fund_AUD


Aggregation Results: 
Sum(Holding/default/PV): 6487038.794313721
Portfolio/default/Name: UK_High_Growth_Equities_Fund_base_fund_GBP


Aggregation Results: 
Sum(Holding/default/PV): 8500960.0
Portfolio/default/Name: UK_High_Growth_Equities_Fund_base_fund_USD


Aggregation Results: 
Sum(Holding/default/PV): 7453050.977933617
Portfolio/default/Name: UK_High_Growth_Equities_Fund_base_fund_EUR


Aggregation Results: 
Sum(Holding/default/PV): 944101101.8223388
Portfolio/default/Name: UK_High_Growth_Equities_Fund_base_fund_JPY




## 11) Add transactions to the base fund

In [45]:
base_fund_transactions = []

base_fund_transactions.append(
    models.TransactionRequest(
        transaction_id=1234567,
        type='FundsIn',
        instrument_identifiers={
            'Instrument/default/Currency': 'USD'},
        transaction_date=holdings_effective_date+timedelta(days=1),
        settlement_date=holdings_effective_date+timedelta(days=1),
        units=10000000,
        transaction_price=models.TransactionPrice(
            price=1,
            type='Price'),
        total_consideration=models.CurrencyAndAmount(
            amount=10000000,
            currency=currency),
        transaction_currency='USD',
        source='Client',
    )
)

# Upsert the transaction into LUSID
response = client.transaction_portfolios.upsert_transactions(
    scope=scope,
    code=base_portfolio_code,
    transactions=base_fund_transactions
    )

## 12) Re-Value the base fund

In [46]:
# Import market data for the day that the holdings of the fund are effective from
prices = pd.read_csv('data/multiplecurrencies-prices2.csv')
prices.head()

,price,type,currency,instrument_name,instrument_figi,instrument_internal
0,1625.78,close,USD,Amazon_Nasdaq_AMZN,BBG000BVPXP1,imd_34634534
1,165.93,close,USD,Apple_Nasdaq_AAPL,BBG000B9XVV8,imd_34535347
2,98.65,close,USD,USTreasury_2.00_2021,BBG00FN3B5K8,imd_34535347
3,143.68,close,USD,USTreasury_6.875_2025,BBG000DQQNJ8,imd_34534539


In [47]:
# Initialise an empty instrument quotes list to hold the quotes
instrument_quotes = []

# Iterate over the quotes
for index, quote in prices.iterrows():
    
    # Add the quote to the list of upsert quote requests
    instrument_quotes.append(
        models.UpsertQuoteRequest(
            quote_id=models.QuoteId(
                provider='DataScope',
                price_source='',
                instrument_id=quote['instrument_figi'],
                instrument_id_type='Figi',
                quote_type='Price',
                price_side='Mid'),
            metric_value=models.MetricValue(
                value=quote['price'],
                unit=quote['currency']),
            effective_at=holdings_effective_date+timedelta(days=1),
            lineage='InternalSystem'
            )
    )

# Upsert the quotes into LUSID
response = client.quotes.upsert_quotes(
    scope=scope,
    quotes=instrument_quotes)

# Pretty print the response
prettyprint.upsert_quotes_response(response)

Quotes Successfully Upserted At: 2019-07-10 13:43:52.031946+00:00


In [48]:
# Create the aggregation request, using the recipe defined earlier
aggregation_request = models.AggregationRequest(
    inline_recipe=inline_recipe,
    effective_at=holdings_effective_date+timedelta(days=1),
    metrics=[
        models.AggregateSpec(key='Holding/default/Cost',
        op='Sum'),
        models.AggregateSpec(key='Holding/default/PV',
        op='Sum'),
        models.AggregateSpec(key='Portfolio/default/Name',
        op='Value'),
    ],
    group_by=[
        'Portfolio/default/Name'
    ])

# Call LUSID to aggregate and value the base fund
response = client.aggregation.get_aggregation_by_portfolio(
    scope=scope,
    code=base_portfolio_code,
    request=aggregation_request)

# Set the total value and total cost of the base fund when it was first created
base_fund_total_value = response.data[0]['Sum(Holding/default/PV)']
base_fund_total_cost = response.data[0]['Sum(Holding/default/Cost)']

prettyprint.heading("Base Fund Total Value", str(base_fund_total_value))
prettyprint.heading("Base Fund Total Cost", str(base_fund_total_cost))

Base Fund Total Value: 52488203.18
Base Fund Total Cost: 53109507.87


In [49]:
# Get the base fund instrument
base_fund_instrument = client.instruments.get_instrument(
    identifier_type='ClientInternal',
    identifier=base_portfolio_code,
    instrument_property_keys=[circulation_property_key])

# Get the unique Lusid Instrument Id from the base fund instrument
luid = base_fund_instrument.lusid_instrument_id
# Get the total circulation from the base fund instrument
base_fund_total_circulation = round(base_fund_instrument.properties[0].value, 0)
# Calculate the unit price of the base fund from the total value and the total circulation
base_fund_unit_price = round(base_fund_total_value / base_fund_total_circulation, 2)

# Create a quote for the unit price of the base fund
instrument_quote = models.UpsertQuoteRequest(
    quote_id=models.QuoteId(
        provider='DataScope',
        price_source='',
        instrument_id=luid,
        instrument_id_type='LusidInstrumentId',
        quote_type='Price',
        price_side='Mid'),
    metric_value=models.MetricValue(
        value=base_fund_unit_price,
        unit='USD'),
    effective_at=holdings_effective_date+timedelta(days=1),
    lineage='InternalSystem'
)

# Call LUSID to upsert the quote
response = client.quotes.upsert_quotes(
    scope=scope,
    quotes=[instrument_quote])

# Pretty print the total value, circulation and unit price of the fund
prettyprint.heading('Base Fund Unit Price', str(base_fund_unit_price))
# Pretty print the response from LUSID

prettyprint.upsert_quotes_response(response)

Base Fund Unit Price: 1312.21
Quotes Successfully Upserted At: 2019-07-10 13:43:57.517343+00:00


## 13) Re-Value the currency portfolios

In [50]:
# Import the FX rates
fx_rates = pd.read_csv('data/multiplecurrencies-fxrates2.csv')
fx_rates.head()

,pair,rate
0,GBP/USD,1.3200
1,AUD/USD,0.7100
2,EUR/USD,1.1700
3,JPY/USD,0.0089


In [51]:
# Prepare an empty list to hold the quote requests
quote_requests = []

# Iterate over each FX rate
for index, fx_rate in fx_rates.iterrows():
    
    # Add a quote for the FX rate
    quote_requests.append(
        models.UpsertQuoteRequest(
            quote_id=models.QuoteId(
                provider='DataScope',
                price_source='',
                instrument_id=fx_rate['pair'],
                instrument_id_type='CurrencyPair',
                quote_type='Rate',
                price_side='Mid'),
            metric_value=models.MetricValue(
                value=fx_rate['rate'],
                unit='rate'),
            effective_at=holdings_effective_date+timedelta(days=1),
            lineage='InternalSystem'))

    # Create the reverse pair name 
    reverse_pair = '/'.join(fx_rate['pair'].split('/')[::-1])

    # Add a quote for the reverse FX rate
    quote_requests.append(
        models.UpsertQuoteRequest(
            quote_id=models.QuoteId(
                provider='DataScope',
                price_source='',
                instrument_id=reverse_pair,
                instrument_id_type='CurrencyPair',
                quote_type='Rate',
                price_side='Mid'),
            metric_value=models.MetricValue(
                value=1/fx_rate['rate'],
                unit='rate'),
            effective_at=holdings_effective_date+timedelta(days=1),
            lineage='InternalSystem'))

# Upsert the quotes into LUSID
response = client.quotes.upsert_quotes(
    scope=scope,
    quotes=quote_requests)

# Pretty print the response
prettyprint.upsert_quotes_response(response)

Quotes Successfully Upserted At: 2019-07-10 13:44:09.313656+00:00


In [52]:
interest_rates = pd.read_csv('data/multiplecurrencies-interestrates2.csv')
interest_rates.head()

,currency,interest_rate
0,USD,0.025300
1,AUD,0.017300
2,EUR,-0.001632
3,JPY,0.001100
4,GBP,0.007708


In [53]:
quote_requests = []

# Iterate over each currency
for currency, currency_portfolio_code in fund_currencies.items():
    
    # Ensure that the currency is not the same as the base fund currency
    if currency!='USD':
        # Get the current FX rate for buying USD from the portfolio currency
        fx_rate = client.quotes.get_quotes(
            scope=scope,
            quote_ids=[
                models.QuoteId(
                    provider='DataScope',
                    price_source='',
                    instrument_id='{}/USD'.format(currency),
                    instrument_id_type='CurrencyPair',
                    quote_type='Rate',
                    price_side='Mid')
            ],
            effective_at=holdings_effective_date+timedelta(days=1)).found[0].metric_value.value
    else:
        # If the currency is the same as the base fund use an fx_rate of 1
        fx_rate = 1
    
    # Get all transactions for the currency portfolio
    response = client.transaction_portfolios.get_transactions(
        scope=scope,
        code=currency_portfolio_code,
        from_transaction_date=holdings_effective_date,
        to_transaction_date=holdings_effective_date,
        filter="type eq 'Buy'")
    
    # Ensure that the at least one transaction exists
    if len(response.values) == 0:
        continue
        
    # Get the hedging transaction which should be the only transaction
    hedge_transaction = response.values[0]
    
    # Get the hedge contract using the instrument id from the transaction
    hedge_contract = json.loads(client.instruments.get_instrument(
        identifier_type='LusidInstrumentId',
        identifier=hedge_transaction.instrument_uid
    ).instrument_definition.content)
    
    # Get the spot rate to buy the portfolio currency
    spot_rate = 1 / fx_rate
    
    # Get the interest rate for USD
    interest_rate_1 = interest_rates.loc[
        interest_rates['currency'] == 'USD', 'interest_rate'].values[0]
    
    # Get the interest rate for the portfolio currency
    interest_rate_2 = interest_rates.loc[
        interest_rates['currency'] == currency, 'interest_rate'].values[0]
    
    # Get the time in days left on the hedge contract
    time = ((parse(timestr=hedge_contract['settlement_date']) \
            - holdings_effective_date+timedelta(days=1)).days) / 365
    
    # Get the strike price of the hedge contract
    strike_price = hedge_contract['strike_price']

    # Work out the forward FX price
    forward_price = spot_rate * math.exp((interest_rate_2 - interest_rate_1) * time)
    
    # Work out the value of the hedge per unit of currency
    value = (forward_price - strike_price) * math.exp(-interest_rate_2 * time)
    
    # Work out the total value of the hedge
    total_value = value * hedge_contract['leg1']['amount']
    
    # Print the value
    prettyprint.heading(
        'USD/' + currency+' swap value', '{} {}'.format(
            round(total_value,2), currency))
    
    print (hedge_transaction.instrument_uid)
    
    # Upsert the quote against the hedging transaction
    quote_requests.append(
        models.UpsertQuoteRequest(
            quote_id=models.QuoteId(
                provider='DataScope',
                price_source='',
                instrument_id=hedge_transaction.instrument_uid,
                instrument_id_type='LusidInstrumentId',
                quote_type='Price',
                price_side='Mid'),
            metric_value=models.MetricValue(
                value=total_value,
                unit=currency),
            effective_at=holdings_effective_date+timedelta(days=1),
            lineage='InternalSystem'))

response = client.quotes.upsert_quotes(
    scope=scope,
    quotes=quote_requests)

prettyprint.upsert_quotes_response(response)

USD/AUD swap value: 166467.83 AUD
LUID_R5VYBQP8
USD/GBP swap value: -52369.8 GBP
LUID_VA4MNK5S
USD/EUR swap value: -198280.58 EUR
LUID_65TL3NNC
USD/JPY swap value: 10249974.49 JPY
LUID_8R4N8DNZ
Quotes Successfully Upserted At: 2019-07-10 13:44:18.329834+00:00


In [56]:
currency_portfolio_values = {}

inline_recipe_complex = inline_recipe
inline_recipe_complex.pricing = models.PricingContext(
    options=models.PricingOptions(
        allow_any_instruments_with_sec_uid_to_price_off_lookup=True))

for currency, currency_portfolio_code in fund_currencies.items():
    aggregation_request = models.AggregationRequest(
        inline_recipe=inline_recipe_complex,
        effective_at=holdings_effective_date+timedelta(days=1),
        metrics=[
            models.AggregateSpec(key='Holding/default/PV',
            op='Sum'),
            models.AggregateSpec(key='Portfolio/default/Name',
            op='Value')
        ],
        group_by=[
            'Portfolio/default/Name'
        ])

    response = client.aggregation.get_aggregation_by_portfolio(
        scope=scope,
        code=currency_portfolio_code,
        request=aggregation_request)

    currency_portfolio_values[currency] = response.data[0]['Sum(Holding/default/PV)']

    prettyprint.aggregation_response_generic(response)

Aggregation Results: 
Sum(Holding/default/PV): 14951932.621817306
Portfolio/default/Name: UK_High_Growth_Equities_Fund_base_fund_AUD


Aggregation Results: 
Sum(Holding/default/PV): 7900418.0744351875
Portfolio/default/Name: UK_High_Growth_Equities_Fund_base_fund_GBP


Aggregation Results: 
Sum(Holding/default/PV): 10497680.0
Portfolio/default/Name: UK_High_Growth_Equities_Fund_base_fund_USD


Aggregation Results: 
Sum(Holding/default/PV): 8774095.48443334
Portfolio/default/Name: UK_High_Growth_Equities_Fund_base_fund_EUR


Aggregation Results: 
Sum(Holding/default/PV): 1189764581.2354827
Portfolio/default/Name: UK_High_Growth_Equities_Fund_base_fund_JPY


